# Demo Complex Process Synchronization
In the Process Synchronization notebook the Synchronization using **start_events** was compared with a sequence of activities. However, the synchronization of two processes e.g. two vessels can not be implemented using sequence, but requires the **start_event** synchronization. 

In this demo two processes each consisting of a sequence of activities representing the processes of two vessels are synchronized, e.g. because they have to make a hand-over of material. 

This basic mechanism is also used in the feeder demo.

In [1]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

## Definition of two sequences of basic activities
The activity names of the first sequence all start with *A Basic activity*. Activity *A Basic activity3* is constrained such that it can only be executed after *B Basic activity 2* has been completed. Thus, in this demo, two processes controlled by a structural control element are synchronized with each other using **start_events**.

In [2]:
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    ID="6dbbbdf7-4589-11e9-bf3b-b469212bff5k",
    registry=registry,
    duration=0,
    keep_resources=keep_resources,
)

sub_processes2 = [
    model.BasicActivity(
        env=my_env,
        name="B Basic activity1",
        ID="5dbbbdf7-4589-11e9-bf3b-b469212bff5b",
        registry=registry,
        duration=1,
        additional_logs=[reporting_activity],
        keep_resources=keep_resources,
    ),
    model.BasicActivity(
        env=my_env,
        name="B Basic activity2",
        ID="5dbbbdf7-4589-11e9-bf3b-b469212bff5c",
        registry=registry,
        duration=500,
        additional_logs=[reporting_activity],
        keep_resources=keep_resources,
    ),
    model.BasicActivity(
        env=my_env,
        name="B Basic activity3",
        ID="5dbbbdf7-4589-11e9-bf3b-b469212bff5d",
        registry=registry,
        duration=120,
        additional_logs=[reporting_activity],
        keep_resources=keep_resources,
    ),
]

activity2 = model.SequentialActivity(
    env=my_env,
    name="B Sequential process",
    ID="5dbbbdf7-4589-11e9-bf3b-b469212bff60",
    registry=registry,
    sub_processes=sub_processes2,
    keep_resources=keep_resources,
)

Process 2 definition

In [3]:
sub_processes = [
    model.BasicActivity(
        env=my_env,
        name="A Basic activity1",
        ID="6dbbbdf7-4589-11e9-bf3b-b469212bff5b",
        registry=registry,
        duration=14,
        additional_logs=[reporting_activity],
        keep_resources=keep_resources,
    ),
    model.BasicActivity(
        env=my_env,
        name="A Basic activity2",
        ID="6dbbbdf7-4589-11e9-bf3b-b469212bff5c",
        registry=registry,
        duration=10,
        additional_logs=[reporting_activity],
        keep_resources=keep_resources,
    ),
    model.BasicActivity(
        env=my_env,
        name="A Basic activity3",
        ID="6dbbbdf7-4589-11e9-bf3b-b469212bff5d",
        registry=registry,
        duration=220,
        additional_logs=[reporting_activity],
        keep_resources=keep_resources,
        start_event=[
            {"type": "activity", "name": "B Basic activity2", "state": "done"}
        ],
    ),
]

activity = model.SequentialActivity(
    env=my_env,
    name="A Sequential process",
    ID="6dbbbdf7-4589-11e9-bf3b-b469212bff60",
    registry=registry,
    sub_processes=sub_processes,
    keep_resources=keep_resources,
)

In [4]:
model.register_processes([activity, activity2, reporting_activity])
my_env.run()

In [5]:
display(plot.get_log_dataframe(reporting_activity, [activity, *sub_processes, activity2, *sub_processes2, reporting_activity]))

,Activity,Timestamp,ActivityState,type,ref
0,Reporting activity,1970-01-01 00:00:00,START,NaN,NaN
1,Reporting activity,1970-01-01 00:00:00,STOP,NaN,NaN
2,B Basic activity1,1970-01-01 00:00:00,START,additional log,5dbbbdf7-4589-11e9-bf3b-b469212bff5b
3,A Basic activity1,1970-01-01 00:00:00,START,additional log,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
4,B Basic activity1,1970-01-01 00:00:01,STOP,additional log,5dbbbdf7-4589-11e9-bf3b-b469212bff5b
5,B Basic activity2,1970-01-01 00:00:01,START,additional log,5dbbbdf7-4589-11e9-bf3b-b469212bff5c
6,A Basic activity1,1970-01-01 00:00:14,STOP,additional log,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
7,A Basic activity2,1970-01-01 00:00:14,START,additional log,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
8,A Basic activity2,1970-01-01 00:00:24,STOP,additional log,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
9,B Basic activity2,1970-01-01 00:08:21,STOP,additional log,5dbbbdf7-4589-11e9-bf3b-b469212bff5c


In [6]:
plot.vessel_planning([
    *sub_processes,
    *sub_processes2
])

Both processes start at the same time. *A Basic activity3* enters waiting state since *A Basic activity2* has not been completed yet.

**Hint:** Having a start_event based on the last activity of a sequence is currently not working. If the constraint is made based on the end of the parent sequence, then the intended synchronization can be accomplished.

When changing the duration of *A Basic activity2* to 1000 then there will be no waiting since the *A Basic activity2* completes after *B Basic activity2* has been completed. 